# Unified LLM Generation Pipeline

This notebook provides an interface to the unified generation pipeline.
All logic lives in `generation.py` - this notebook is for interactive testing and batch runs.

The pipeline handles both Paleolithic and Holocene people automatically based on the sampled birth year.

In [1]:
import dill
from tqdm import tqdm

from generation import (
    generate_person,
    generate_batch,
    generate_batch_parallel,
    # Individual steps if needed
    generate_geography,
    generate_demographics,
    generate_structured_incidents,
    generate_historical_context,
    generate_name,
    generate_narrative,
    quality_check,
    reset_to_stage
)
from llm_utils import GenerationContext, extract_json

from person import sample_person, Person

import copy

# Batch Generation

In [2]:
#sample_person()

Warning 3: Cannot find header.dxf (GDAL_DATA is not defined)


In [6]:
test_people_2 = generate_batch_parallel(n=30, model="gpt-5.2", workers=30)

Sampling 30 people...
Generating 30 people with 30 parallel workers...


100%|███████████████████████████████████████████| 30/30 [03:55<00:00,  7.84s/it]

Done: 30 generated


In [7]:
new_tests = test_people + test_people_2

In [8]:
with open('test_examples.pkl', 'wb') as f:
    dill.dump(new_tests,f)

In [9]:
%run export.py test_examples.pkl

Loading people from test_examples.pkl...
Found 40 people
Removing existing markdown files from ../_lives...
  Removed 10 files
  Exported 10 people...
  Exported 20 people...
  Exported 30 people...
  Exported 40 people...

Successfully exported 40 people to ../_lives


## Step-by-Step Generation

For debugging or custom workflows, you can run each step individually.